In [58]:
from itertools import combinations, chain
import random
import string

def chain_unique(*args):
    seen = set()
    for v in chain(*args):
        if v not in seen:
            seen.add(v)
            yield v

def subsequence_generator(sequence):
    for ss in combinations(sequence, len(sequence) - 1):
        yield ''.join(ss)

def get_subsequences(amino_acid_keys):
    for sequence in amino_acid_keys:
        yield from subsequence_generator(sequence)

def process_tuple(a, b):
    result = a + b
    return result

def map_function_over_tuples(sequence):
    chunk = []
    for i, item in enumerate(sequence):
        chunk.append(item)
        if i % 1000 == 0 and i > 0:  # process in chunks of 1000
            yield from map(process_tuple, chunk)
            chunk = []
    yield from map(process_tuple, chunk)  # process the remaining items

amino_acid_keys = [''.join(random.choices(string.ascii_uppercase[:4], k=3)) for _ in range(1000)]
subsequences = chain_unique(get_subsequences(amino_acid_keys))

result = map_function_over_tuples(subsequences)
list(result)

TypeError: process_tuple() missing 1 required positional argument: 'b'

In [47]:
from itertools import combinations, chain
import random
from collections import Counter
from more_itertools import ichunked

def chain_unique(*args):
    seen = set()
    for v in chain(*args):
        if v not in seen:
            seen.add(v)
            yield v

# subsequence generator of the sequence's length - 1
def subsequence_generator(sequence):
    for ss in combinations(sequence, len(sequence) - 1):
        yield ''.join(ss)

# generate a random 10 sequences of length 5
def get_subsequences(amino_acid_keys):
    for sequence in amino_acid_keys:
        yield from subsequence_generator(sequence)

amino_acid_keys = [''.join(random.choices(string.ascii_uppercase[:4], k=3)) for _ in range(1000)]
subsequences = chain_unique(get_subsequences(amino_acid_keys))
#
# counter = Counter()
#
# chunk_size = 100  # Define a reasonable chunk size here based on your memory availability
# counters= []
# for chunk in ichunked(subsequences, chunk_size):
#     counters.append(Counter(chunk))

# remove subsequences with a count of 1
# counter = {k:v for k,v in counter.items() if v > 1}
list(subsequences)

['BD',
 'BB',
 'DB',
 'DA',
 'AA',
 'BC',
 'CD',
 'DD',
 'CA',
 'CC',
 'AC',
 'AD',
 'CB',
 'BA',
 'AB',
 'DC']

In [24]:
# drop 1 value


{'AA': 2,
 'AC': 2,
 'CC': 2,
 'DC': 2,
 'CB': 2,
 'CA': 2,
 'BA': 3,
 'BD': 2,
 'BC': 2}

In [49]:
c = Counter(['a', 'b', 'c', 'a', 'b', 'b'])
c

Counter({'a': 2, 'b': 3, 'c': 1})

In [53]:
# join directories with collections library
from collections import ChainMap


# create a chainmap of the current directory and the home directory
chainmap = from collections import defaultdict
from operator import itemgetter

from itertools import chain, combinations, count, filterfalse, repeat
from more_itertools import all_equal, pairwise, prepend, split_when



def levenshtein(sequence_list):
    '''
    Given a list of sequences, return a list of sets of sequences that are all identical except for one character.

    Args:
    sequence_list: a list of sequences to be compared
    Returns:
    A list of sets of sequences that are all identical except for one character.
    '''

    def hash_subsequences(seq):
        sub = prepend(tuple(seq), combinations(seq, len(seq) - 1))
        return zip(map(hash, sub), count(), repeat(seq))

    def hash_clusters(groups):
        clusters = [(c[0][1], set(map(itemgetter(-1), c)))
                    for c in split_when(groups, lambda s1, s2: s1[1] != s2[1])]
        if clusters[0][0] == 0:
            clusters = [(p, c.union(clusters[0][1])) for p, c in clusters[1:]]
        return filterfalse(lambda g: len(g[1]) == 1, clusters)

    sub_keys = chain(*map(hash_subsequences, sequence_list))
    sorted_keys = sorted(sub_keys, key=lambda k: k[0] + k[1])

    hash_groups = split_when(sorted_keys, lambda s1, s2: s1[0] != s2[0])
    hash_groups = filterfalse(lambda g: len(g) == 1, hash_groups)
    hash_groups = chain(*map(hash_clusters, hash_groups))

    return set(tuple(sorted(edge)) for group in hash_groups for edge in combinations(group[1], 2))

clusters = levenshtein(amino_acid_keys[:100000])
clusters[0:2]({'a': 1, 'b': 2}, {'b': 3, 'c': 4})
# print the joined chainmap
print(chainmap)

ChainMap({'a': 1, 'b': 2}, {'b': 3, 'c': 4})


In [54]:
from collections import defaultdict
from operator import itemgetter

from itertools import chain, combinations, count, filterfalse, repeat
from more_itertools import all_equal, pairwise, prepend, split_when



def levenshtein(sequence_list):
    '''
    Given a list of sequences, return a list of sets of sequences that are all identical except for one character.

    Args:
    sequence_list: a list of sequences to be compared
    Returns:
    A list of sets of sequences that are all identical except for one character.
    '''

    def hash_subsequences(seq):
        sub = prepend(tuple(seq), combinations(seq, len(seq) - 1))
        return zip(map(hash, sub), count(), repeat(seq))

    def hash_clusters(groups):
        clusters = [(c[0][1], set(map(itemgetter(-1), c)))
                    for c in split_when(groups, lambda s1, s2: s1[1] != s2[1])]
        if clusters[0][0] == 0:
            clusters = [(p, c.union(clusters[0][1])) for p, c in clusters[1:]]
        return filterfalse(lambda g: len(g[1]) == 1, clusters)

    sub_keys = chain(*map(hash_subsequences, sequence_list))
    sorted_keys = sorted(sub_keys, key=lambda k: k[0] + k[1])

    hash_groups = split_when(sorted_keys, lambda s1, s2: s1[0] != s2[0])
    hash_groups = filterfalse(lambda g: len(g) == 1, hash_groups)
    hash_groups = chain(*map(hash_clusters, hash_groups))

    return set(tuple(sorted(edge)) for group in hash_groups for edge in combinations(group[1], 2))

clusters = levenshtein(amino_acid_keys[:100000])
clusters[0:2]

TypeError: 'set' object is not subscriptable

time and memory profiling

In [ ]:
import numpy as np
from operator import getitem, itemgetter
from itertools import combinations, count, groupby, repeat, starmap
from more_itertools import unique_justseen

def levenshtein_cluster(sequences):
    hashes = np.array([(hash(s[:i] + s[i + 1:]) ^ (len(s) - i), si)
                       for si, s in enumerate(sequences) for i in range(len(s) + 1)])

    hashes = hashes[hashes[:, 0].argsort(), :]

    unique_h, index = np.unique(hashes[:, 0], return_index=True)
    index = [index[i] for i in range(1, len(unique_h)) if
             abs(unique_h[i] - unique_h[i - 1]) > 10]  # CHANGE TO MAX SEQUENCE LENGTH INSTEAD OF 10

    clusters = []
    for grouped_i in np.split(hashes, index, axis=0):

        cluster = [set(sequences[i] for i in map(itemgetter(1), g))
                   for _, g in groupby(grouped_i.tolist(), key=itemgetter(0))]

        if len(set(map(len, set.union(*cluster)))) == 2:
            gap = min(set.union(*cluster), key=len)
            for i in range(len(cluster)): cluster[i].add(gap)

        clusters += [tuple(sorted(e)) for c in cluster if len(c) > 1 for e in combinations(c, 2)]

    return clusters

clusters = levenshtein_cluster(amino_acid_keys)
clusters[0:2]

In [17]:
%%timeit
import numpy as np
np.random.seed(0)
np.random.rand(1000000)

6.76 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
